# 03b - Vertex AI + BQML - Online Predictions with BQML Models

Models built with BigQuery ML (BQML), like the one in (03a), can also be exported for use and deployment outside of BigQuery.  A Vertex AI Endpoint can be used for online predictions with an exported model.  This demonstration shows the process of exporting and deploying a BQML model with Vertex AI.

### Video Walkthrough of this notebook:
Includes conversational walkthrough and more explanatory information than the notebook:

<p><center><a href="https://youtu.be/7y_t_bW0LHQ" target="_blank" rel="noopener noreferrer"><img src="architectures/thumbnails/playbutton/03b.png" width="50%"></a></center></p>

### Prerequisites:
-  03a - BigQuery Machine Learning (BQML) - Machine Learning with SQL

### Overview:
-  Export the BigQuery ML model built in (03a) to a GCS bucket URI
   -  EXPORT MODEL …
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Upload Model
      -  Model - aiplatform.Model.upoad
   -  Create Endpoint
      -  Endpoint - aiplatform.Endpoint.create
   -  Deploy to Endpoint
      -  Endpoint.deploy(model=Model)
   -  Online Predictions
      -  Endpoint.predict

### Resources:
-  [Export formats for BigQuery ML models](https://cloud.google.com/bigquery-ml/docs/exporting-models)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)


---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/03b_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/03b_console.png">

---
## Setup

inputs:

In [1]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'fraud'
NOTEBOOK = '03b'

# Resources
DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [2]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [4]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
params = {"URI": URI}
DIR = f"temp/{NOTEBOOK}"

environment:

In [5]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Export the BigQuery Model

Export the BigQuery Model:
- https://cloud.google.com/bigquery-ml/docs/exporting-models

In [6]:
export = bigquery.query(query = f"EXPORT MODEL {DATANAME}.{DATANAME}_lr OPTIONS(URI = '{URI}')")

In [7]:
export.result()

---
## Serving

### Upload The Model
https://googleapis.dev/python/aiplatform/latest/aiplatform.html?highlight=aiplatform%20model%20upload#google.cloud.aiplatform.Model.upload

In [8]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = URI,
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/764015827198/locations/us-central1/models/6040246292024655872/operations/8528026658517549056
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/764015827198/locations/us-central1/models/6040246292024655872
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/764015827198/locations/us-central1/models/6040246292024655872')


In [9]:
model.display_name

'03b_fraud_20211129141713'

### Create An Endpoint

In [10]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/764015827198/locations/us-central1/endpoints/714251549496311808/operations/1853692010754473984
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/764015827198/locations/us-central1/endpoints/714251549496311808
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/764015827198/locations/us-central1/endpoints/714251549496311808')


In [11]:
endpoint.display_name

'03b_fraud_20211129141713'

### Deploy Model To Endpoint

In [12]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/764015827198/locations/us-central1/models/6040246292024655872 to Endpoint : projects/764015827198/locations/us-central1/endpoints/714251549496311808
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/764015827198/locations/us-central1/endpoints/714251549496311808/operations/5365373820196618240
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/764015827198/locations/us-central1/endpoints/714251549496311808


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [13]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [14]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,79470,1.073063,-0.026793,0.796502,1.754389,-0.219910,0.930349,-0.515894,0.461495,-0.132966,...,0.048539,-0.258218,0.171861,1.028847,-0.037692,-0.013125,0.0,0,07fdced0-3837-47a1-9526-64d74ad9b113,TEST
1,122082,1.878563,0.020785,-1.621113,2.908813,2.507847,4.709442,-0.830626,1.136154,-0.395755,...,0.121098,0.707538,0.140100,0.155684,0.016375,-0.053892,0.0,0,7c1f61ba-7586-414e-ba8a-1c4385d59933,TEST
2,11494,-1.169744,0.462878,1.587579,-1.252560,1.012817,-0.918413,0.817632,-0.522804,1.388247,...,-0.042144,-0.021269,-0.283979,0.556661,-0.191722,-0.071773,0.0,0,a3046c87-cee5-40fd-9302-4d230b823246,TEST
3,58900,-2.728403,-2.340346,2.551039,2.623092,3.014728,-0.807294,-2.286621,0.757561,-1.279297,...,0.370139,-0.212784,-0.104956,1.355781,-0.053063,0.125386,0.0,0,bc3f2800-a4bb-4077-b017-f55f03c4f00c,TEST


In [15]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET,'splits'])]].to_dict(orient='records')[0]
#newob

In [16]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [17]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[{'predicted_Class': ['0'], 'Class_probs': [0.0990060111289882, 0.9009939888710118], 'Class_values': ['1', '0']}], deployed_model_id='5358623849594224640', explanations=None)

In [18]:
prediction.predictions[0]#['classes'][np.argmax(prediction.predictions[0]['scores'])]

{'predicted_Class': ['0'],
 'Class_probs': [0.0990060111289882, 0.9009939888710118],
 'Class_values': ['1', '0']}

In [19]:
prediction.predictions[0][f'{VAR_TARGET}_values'][np.argmax(prediction.predictions[0][f'{VAR_TARGET}_probs'])]

'0'

### Get Predictions: REST

In [20]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [21]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    {
      "Class_probs": [
        0.0990060111289882,
        0.90099398887101179
      ],
      "predicted_Class": [
        "0"
      ],
      "Class_values": [
        "1",
        "0"
      ]
    }
  ],
  "deployedModelId": "5358623849594224640",
  "model": "projects/764015827198/locations/us-central1/models/6040246292024655872",
  "modelDisplayName": "03b_fraud_20211129141713"
}


### Get Predictions: gcloud (CLI)

In [22]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[{'Class_probs': [0.0990060111289882, 0.9009939888710118], 'Class_values': ['1', '0'], 'predicted_Class': ['0']}]


---
## Remove Resources
see notebook "99 - Cleanup"